## Pre-requisite:
1. Generate 2 Ethereum testnet addresses (address_1, address_2)
2. Keep Address_1 as your main test address (use the Sepolia test network).

In [2]:
#Defining Libraries:
import os
import ecdsa
from Cryptodome.Hash import keccak


def sha3(a):
    keccak_hash = keccak.new(digest_bits=256)
    keccak_hash.update(a)
    return '0x' + keccak_hash.hexdigest()[24:]

#Function for Checksum
def checksum_encode(addr_str): # Takes a hex (string) address as input
    keccak_hash = keccak.new(digest_bits=256)
    out = ''
    addr = addr_str.lower().replace('0x', '')
    keccak_hash.update(addr.encode('ascii'))
    hash_addr = keccak_hash.hexdigest()
    for i, c in enumerate(addr):
        #print(i,c, hash_addr[i])
        if int(hash_addr[i], 16) >= 8:
            out += c.upper()
        else:
            out += c
    return '0x' + out


def test(addrstr):
    assert(addrstr == checksum_encode(addrstr))

def ecdsa_maths(private_key_from_hex):
    sk = ecdsa.SigningKey.from_string(private_key_from_hex, curve = ecdsa.SECP256k1)
    #actual publickey to verify messages that singed using own pvt key
    verification_key = sk.verifying_key
    #making Public Key:
    public_key =  verification_key.to_string()
    #print(public_key.hex())
    return public_key

def address(passwrd):
    address = sha3(ecdsa_maths(passwrd))
    #print("Raw_Address", address,'\n')
    print(f'\nPrivate Key: {passwrd.hex()}\nChecksum Address: {checksum_encode(address)}')
    return checksum_encode(address), passwrd.hex()

In [3]:
def random_key():
        private_key = os.urandom(32).hex()
        return bytes.fromhex(private_key)

password = b'\x1e:\x0e\xa1\xf0)\x86rE\xbe\x21\x97\x13)\xfa\xabQ\xc2\xfc\xf8D\xb2\x8b\xb7b"\x01\xbe\xe61\xf1\xca'
address_1, private_key1 = address(password)
password = b'\x9be\xcc\xb1\x80\xd3\xaf\x96\x31\x84\xae\xe6\xf1Wf\x80\xfc\x00\xe8w\xa6\xb8\x02-\xd4\xdc\r\x163G\xf97'
address_2, private_key2 = address(password)


Private Key: 1e3a0ea1f029867245be21971329faab51c2fcf844b28bb7622201bee631f1ca
Checksum Address: 0xBD813Ae2347e85F0723A83582AeDc08412B8B243

Private Key: 9b65ccb180d3af963184aee6f1576680fc00e877a6b8022dd4dc0d163347f937
Checksum Address: 0xA5403C040D9b7de2CD3bF0974A4516219407b88D


## Task:
* (1 point) Create a user-interacting interface to connect to different blockchain networks, i.e., mainnet, testnets (any two), and ganache (Hint: Take input from the User and use the if-else method to connect to different networks. User different Infura/Alchemy Endpoints).

In [4]:
from web3 import Web3

class Interface:

    def connect_to_web(self):
        print("\nProvide a link to a network")

        testnet_url = str(input())
        '''
        if net =='main':
            testnet_url = 'https://eth-mainnet.g.alchemy.com/v2/RBkI6V3iQvUhearJcH5muyoMJwxJu7Lp'
        elif net =='goerli':
            testnet_url = 'https://goerli.infura.io/v3/82d61aeb5732496aa0a4d98e9d82dbe5'
        elif net =='sepolia':
            testnet_url = 'https://eth-sepolia.g.alchemy.com/v2/zcri9jBzeaPSwr9yi2mkhz1aW4qEB6GQ'
        elif net == 'ganache':
            testnet_url = 'http://127.0.0.1:7545'
        '''
        web3 = Web3(Web3.HTTPProvider(testnet_url))
        print("Connected to Testnet:",web3.is_connected())
        return web3

* (1 point) Get some Ethereum test coins to your testnet address_1 from faucet B. Provide the Transaction link of the testnet work.

https://sepolia.etherscan.io/tx/0x582dc67d900a16c6e6484a702f213a04c0438a87e0c6e427d0268bd68d626a9d

* (2 points) From Task_1, connect to the Ethereum network and provide codes to interact with the blockchain (Testnet will be better):
 1. Checking block height at any time.
 2. Getting information about a block.
 3. Get balance information of your address_1.
 4. From Task 4.3. Convert the balance to ether.
(Hint: https://web3js.readthedocs.io/en/v1.2.4/Links to an external site. or https://web3py.readthedocs.io/Links to an external site.)

In [5]:
infs = Interface()
web3_connect = infs.connect_to_web()


Provide a link to a network
Connected to Testnet: True


In [25]:
#block height
web3_connect.eth.get_block('latest')['number']

5348304

In [26]:
#info about the block
web3_connect.eth.get_block(5328313)

AttributeDict({'baseFeePerGas': 2154694891,
 'blobGasUsed': '0xc0000',
 'difficulty': 0,
 'excessBlobGas': '0x4b40000',
 'extraData': HexBytes('0x4e65746865726d696e64'),
 'gasLimit': 30000000,
 'gasUsed': 13837781,
 'hash': HexBytes('0x90d060505ee20e9471f79182f4507a31ed0398b654b3729492d333097265e930'),
 'logsBloom': HexBytes('0x8824300ec06020c820598e020018e552009940480d11200342940ecac03943306248002080200409bd18ab04142c0a832422004232a2c324951012c0a42400245a00980390cf8103e00845ba01000a8cc40058849c85605a4400a804404142208a657439537163b8e69c42470500382585092cc0010091902180865400480f1000804408c3a0950c32015d30084810025001048c8081280459105810075635684e0a02b208a40ce2c0404e400000022c0a5040184104b820249d28200c00009480901043860800100441020b0c1aa015c0b3141010cb40040bb0b4dc80826009155401048e8824b0c48eb88060c00884080202b82a320000d060320480205225'),
 'miner': '0x1268AD189526AC0b386faF06eFfC46779c340eE6',
 'mixHash': HexBytes('0xac4a91f52476dfbfbf43db72653f83736feb60af0190b718f5c820506f8f058b'),
 'nonc

In [27]:
#obtaining balance in ether
balance_in_Wei = web3_connect.eth.get_balance(address_1)
web3_connect.from_wei(balance_in_Wei, 'ether')

Decimal('0.4429')

* (1 point) Send some Eth test coins from address_1 to address_2 and provide Transaction_ID. (Use seminar file, not Metamask)

In [28]:

tx = {
      'nonce':web3_connect.eth.get_transaction_count(address_1),
      'to': address_2,
      'value': web3_connect.to_wei(float(0.05),'ether'),
      'gas':200000,
      'gasPrice':web3_connect.to_wei('50','gwei')
      }

signed_tx = web3_connect.eth.account.sign_transaction(tx,private_key1)

tx_hash = web3_connect.eth.send_raw_transaction(signed_tx["rawTransaction"]).hex()
print('Transaction_id:', tx_hash)

Transaction_id: 0xb967e9ef81cad5e6b478914a3ffbd101b1a8ea43f5a56b928848db1c89c7f912


* (2 points) Create a smart contract to add two numbers. (Hint:
1. Use ide http://remix.ethereum.org/Links to an external site..
2. Use any version of the compiler.
3. Create a function add (passing two parameters from User) variables declared in the state variable.
4. Connect to any blockchain.
5. Just provide the code.
6. Reference document: https://docs.soliditylang.org/en/v0.8.24/).

pragma solidity >= 0.8.0;

contract summation{
    uint public val=1;

    function add(uint number_1, uint number_2) public returns(uint){
        val = number_1+number_2;
        return val;
    }
}


* (3 points) Create an ERC20 smart contract:
1. Create 100 tokens.
2. Deploy the contract from address_2. Provide contract creation transaction_id and contract address (the address created when the contract is deployed) and a 3-letter abbreviation of your coin. We will check the existence of the sepolia testnet.
3. Send ten tokens to 0x80773c866332a7f3CFdD583d0b3A96655F035d6C address. (Provide Transaction Id)

// contracts/GLDToken.sol
// SPDX-License-Identifier: MIT


pragma solidity ^0.8.0;

import "@openzeppelin/contracts/token/ERC20/ERC20.sol";

contract SampleErc is ERC20 {
    constructor(string memory TokenName, string memory TokenSymbol) ERC20(TokenName, TokenSymbol) {
        _mint(msg.sender, 100);
    }

    function decimals() public view virtual override returns (uint8) {
    return 0;
    }
}



Contract creation transaction id: 0x8e7200305e325ed2994b48970c84b312d95ce0ac36cf6157a7e4004410afb520

Contract address: 0x4278e1f857eb0573fb1aa0e38f8c7f816a6d6a2d

Abbreviation: FST

Transaction id: 0x574e1c4f82450a71e8304e55a898cdb9ccfb9495760f7520d1d28ff38e53e078